# NLP analysis objectives
- switch to outscraper to extract full information of reviews ✓
- check if the place might have water army?
- extract the most related
- use sentiment analysis to analysis the score rather than using google score
- NLTK

# To Do:
- download a set of data with only reviews and rating or Aurther ✓
- Word2vec to show its true reviews
- sentiment analysis to measure its true value

# Tech issue
- maybe code a function to download necessary nlp trained model

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('data_references/Restaurant_Reviews.tsv',sep='\t')


In [3]:
from gensim.models import Word2Vec, KeyedVectors

In [4]:
model = KeyedVectors.load_word2vec_format('data_references/GoogleNews-vectors-negative300.bin', binary=True,limit=1000)